In [1]:
! pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
! pip install transformers[torch]

In [3]:
! pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1


In [4]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd

In [5]:
# Load Dataset1
df = pd.read_csv('preprocessed_dataset1.csv')

In [6]:
df.head()

,Content,Preprocessed_Content
0,All human beings are born free and equal in di...,human being born free equal dignity right endo...
1,Everyone is entitled in full equality to a fai...,everyone entitled full equality fair public he...
2,- Everyone charged with a penal offence has th...,everyone charged penal offence right presumed ...
3,No one shall be subjected to arbitrary interfe...,one shall subjected arbitrary interference pri...
4,- Everyone has the right to freedom of movemen...,everyone right freedom movement residence with...


In [7]:
from transformers import DistilBertTokenizer, DistilBertModel
import pandas as pd
import torch

# Load pre-trained DistilBERT tokenizer and model
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [8]:
# Load and preprocess Dataset1
df = pd.read_csv('preprocessed_dataset1.csv')
df['Content'] = df['Content'].astype(str)
df['Preprocessed_Content'] = df['Preprocessed_Content'].astype(str)

# Tokenize and create embeddings
def create_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# Generate embeddings for all human rights articles
embeddings = create_embeddings(df['Content'].tolist())

# Save embeddings
torch.save(embeddings, 'human_rights_embeddings.pt')
df.to_csv('human_rights_with_embeddings.csv', index=False)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

def find_most_relevant_right(query, embeddings, df):
    query_embedding = create_embeddings([query])
    similarities = cosine_similarity(query_embedding, embeddings)
    most_similar_idx = similarities.argmax()
    return df.iloc[most_similar_idx]['Content']

# Load precomputed embeddings
embeddings = torch.load('human_rights_embeddings.pt')
df2 = pd.read_csv('human_rights_with_embeddings.csv')

<ipython-input-11-01e0e79f35d1>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load('human_rights_embeddings.pt')


In [14]:
# Example query
query = "My neighbor murdered my wife. what should i do ?"
relevant_right = find_most_relevant_right(query, embeddings, df)
print(relevant_right)

1. All persons shall be equal before the courts and tribunals. In the determination of any criminal charge against him, or of his rights and obligations in a suit at law, everyone shall be entitled to a fair and public hearing by a competent, independent and impartial tribunal established by law. The press and the public may be excluded from all or part of a trial for reasons of morals, public order (ordre public) or national security in a democratic society, or when the interest of the private lives of the parties so requires, or to the extent strictly necessary in the opinion of the court in special circumstances where publicity would prejudice the interests of justice; but any judgement rendered in a criminal case or in a suit at law shall be made public except where the interest of juvenile persons otherwise requires or the proceedings concern matrimonial disputes or the guardianship of children.
2. Everyone charged with a criminal offence shall have the right to be presumed innoce

In [12]:
# Example query
query = "What is the right to health?"
relevant_right = find_most_relevant_right(query, embeddings, df)
print(relevant_right)

The right to water is vital for every living thing – without it we would die. We need water to drink and clean ourselves with, and for cooking and sanitation, which is also a right. Water is essential in the enjoyment of all other human rights including the right to life. It is also key in food production.

The right to sanitation enables people to access a clean and healthy environment, without it there are barriers to other rights, such as health and education.

Water must be available and accessible to all people in sufficient quantity and of adequate quality for all of the reasons it is needed. However, there are many ways in which the rights to water and sanitation are violated.

Pollution can make water unsafe. Water scarcity can lead to devastating drought. The climate crisis can often make this worse, exacerbating the lack of accessible, clean water. Governments and companies might control water supplies in ways which ensure people do not have adequate access to it. Human right

In [26]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Load the dataset
df2 = pd.read_csv('/content/preprocessed_dataset2.csv')
df2['Title'] = df2['Title'].astype(str)
df2['Description'] = df2['Description'].astype(str)
df2['Preprocessed_Description'] = df2['Preprocessed_Description'].astype(str)

# Convert Title to categorical labels
df2['Label'] = df2['Title'].astype('category').cat.codes
label_mapping = dict(enumerate(df2['Title'].astype('category').cat.categories))

# Convert the DataFrame to a Dataset
dataset = Dataset.from_pandas(df2[['Description', 'Label']])

# Load tokenizer and model
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_mapping))

# Preprocess the dataset
def preprocess_function(examples):
    # Tokenize the inputs and add labels to the output
    tokenized_inputs = tokenizer(examples['Preprocessed_Description'], truncation=True, padding='max_length', max_length=512)
    tokenized_inputs['labels'] = examples['Label']
    return tokenized_inputs

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and evaluation sets
train_df, eval_df = train_test_split(df[['Preprocessed_Description', 'Label']], test_size=0.2, random_state=42)

# Convert them to Dataset objects
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

# Tokenize the datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# Training arguments with evaluation strategy
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize Trainer with eval_dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,  # Add evaluation dataset
)

# Fine-tune the model
trainer.train()


Map:   0%|          | 0/157 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,5.269656
2,No log,5.291710
3,No log,5.300957


TrainOutput(global_step=60, training_loss=5.227486673990885, metrics={'train_runtime': 1599.7307, 'train_samples_per_second': 0.294, 'train_steps_per_second': 0.038, 'total_flos': 62601327728640.0, 'train_loss': 5.227486673990885, 'epoch': 3.0})

In [29]:
# Save the model and tokenizer
model.save_pretrained('./fine-tuned-distilbert-law')
tokenizer.save_pretrained('./fine-tuned-distilbert-law')

# Save the label mapping for later use
import json
with open('label_mapping.json', 'w') as f:
    json.dump(label_mapping, f)

print("Fine-tuning completed and model saved to 'fine-tuned-distilbert-law'")

Fine-tuning completed and model saved to 'fine-tuned-distilbert-law'
